# import data

In [2]:
from DataCollector import importNotDetectData

data = importNotDetectData(2101, 2201)
print('Total size of data is... %d' % len(data))

ConnectionError: ConnectionError(('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))) caused by: ProtocolError(('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)))

- Categorize data by its path
- Export list of signature

In [1]:
from DataGrouper import groupBySignature

#ここでは、同じsignature内に同一のipからのアクセスがどれくらいあるか確認する。
dataS = groupBySignature(data)
# result1はsignatureごとに分類した上に同一destination_ipについて整理したもの
result1 = {}
for s, hits in dataS.items():
    #それぞれのIPについて分類する。
    resultIps = {}
    for hit in hits:
        if not hit['_source']['destination_ip'] in resultIps:
            resultIps[hit['_source']['destination_ip']] = []
        resultIps[hit['_source']['destination_ip']].append(hit)
    # Signature s についての分類が完了
    # データとして格納
    result1[s] = resultIps
# result1を並び変える。
# len(x[1])refers to the size of the variation of destination_ip
# ↑ if this value is bigger than 1, it means that it is prevalent
result1 = sorted(result1.items(), key=lambda x: len(dataS[x[0]]), reverse=True)
result1 = sorted(result1, key=lambda x: len(x[1]), reverse=True)
# Print and Export result1 as "signatures_by_num_destIP(1-10).csv"
f = open("resultData/signatures_by_num_destIP(2021.1.1-).csv", "wb")
# 書き出す項目について整理しておく
print("signature".center(50), "Spots", "Counts")
f.write('Signature,Spots,Hits\n'.encode('utf-8'))
for s, r in result1:
    print(s[:50].ljust(50), str(len(r)).rjust(5), str(len(dataS[s])).rjust(6))
    b = (s + ',' + str(len(r)) + ',' + str(len(dataS[s])) + '\n').encode('utf-8')
    f.write(b)
f.flush()
f.close()

NameError: name 'data' is not defined

In [ ]:
from search.VulnersSearch import search as vSearch
for s, d in dataS.items():
    vSearch(s)